In [1]:
import os
import glob
import shutil
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 랜덤 시드
np.random.seed(2016)

In [3]:
# 원본 데이터root_path
r_path = 'data/101_ObjectCategories'

In [4]:
# 이미지들이 배치될 폴더의 이름
o_path = 'data/Caltech-101'

In [5]:
# 읽어올 파일 목록을 만든다.
path = f'{r_path}/*/*.jpg'
# glob.glob(path)
files = sorted(glob.glob(path))
files = np.array(files)
files

array(['data/101_ObjectCategories\\BACKGROUND_Google\\image_0001.jpg',
       'data/101_ObjectCategories\\BACKGROUND_Google\\image_0002.jpg',
       'data/101_ObjectCategories\\BACKGROUND_Google\\image_0003.jpg',
       ..., 'data/101_ObjectCategories\\yin_yang\\image_0058.jpg',
       'data/101_ObjectCategories\\yin_yang\\image_0059.jpg',
       'data/101_ObjectCategories\\yin_yang\\image_0060.jpg'],
      dtype='<U58')

In [6]:
# 결과 데이터 레이블(폴더이름)
use_labels = ['airplanes', 'Motorbikes', 'Faces_easy', 
              'watch', 'Leopards', 'bonsai']


# 각 레이블 별 이미지 개수를 담을 리스트
labels_count = [0, 0, 0, 0, 0, 0]

In [7]:
# 학습, 평가, 테스트에 사용할 이미지 건수
train_nums = [80, 80, 44, 24, 20, 13]
valid_nums = [80, 80, 44, 24, 20, 13]
test_nums = [640, 638, 348, 191, 160, 102]

In [8]:
for i in range(0, len(use_labels)):
    # 폴더가 존재하지 않으면 만든다.
    if not os.path.exists(f'{o_path}/train_org/{i}'):
        os.makedirs(f'{o_path}/train_org/{i}')
    
    if not os.path.exists(f'{o_path}/test/{i}'):
        os.makedirs(f'{o_path}/test/{i}')

In [9]:
# 파일 수 만큼 처리
for f1 in files :
    # 파일이름을 가져온다.
    filename = os.path.basename(f1)
    # print(filename)
    
    # 파일들이 들어 있는 폴더의 이름을 가져온다.
    a1 = os.path.split(f1)
    a2 = os.path.split(a1[0])
    parent_dir = a2[1]
    # print(parent_dir)
    
    # 위에서 구한 폴더 이름이 우리가 사용하고자 하는 폴더(use_labels)
    # 에 포함되어 있는 것만 추출해 복사한다.
    if parent_dir in use_labels :
        # 폴더 이름이 use_labels에서 몇번째 인지 확인한다.
        ind = use_labels.index(parent_dir)
        # print(ind)
        
        # 해당 폴더의 이미지 수 개수를 가져온다.
        num = labels_count[ind]
        valid_num = valid_nums[ind]
        test_num = test_nums[ind]
        
        # 아직 복사할 이미지가 남았다면..
        # 학습용과 검증용 이미지 복사
        if num < train_nums[ind] + valid_nums[ind] :
            # 복사한다
            cp_path = f'{o_path}/train_org/{ind}'
            shutil.copy(f1, cp_path)
            
        # 테스트용 이미지 복사
        else :
            cp_path = f'{o_path}/test/{ind}'
            shutil.copy(f1, cp_path)
        
        # 복사한 이미지수를 증가시킨다.
        labels_count[ind] += 1
        
    else :
        # 사용할 레이블과 폴더이름이 다르면 건너뛴다.
        continue
        
print('복사 완료')

복사 완료


In [10]:
# train 데이터를 train, valid 로 나눈다.
# 홀드아웃의 수 만큼(2회) 반복한다.
for ho in range(0, 2) :
    # 만약 폴더가 존재하지 않으면 생성한다.
    for ii in range(0, len(use_labels)) :
        if not os.path.exists(f'{o_path}/train/{ho}/{ii}') :
            os.makedirs(f'{o_path}/train/{ho}/{ii}')
        if not os.path.exists(f'{o_path}/valid/{ho}/{ii}') :
            os.makedirs(f'{o_path}/valid/{ho}/{ii}')
            
        # 데이터를 읽어들인다.
        # 경로를 train_org폴더안에 숫자폴더 내의 모든 이미지파일로..
        path = f'{o_path}/train_org/{ii}/*.jpg'
        # 해당 경로의 이미지 파일 목록을 가져온다.
        files = sorted(glob.glob(path))
        # 경로를 ndarray로 생성한다.
        files = np.array(files)
        # 파일의 순서를 나타내는 인덱스를 랜덤하게 섞어 준다
        perm = np.random.permutation(len(files))
        # 파일목록을 랜덤하게 섞어준다.
        random_train = files[perm]
        # 학습용과 검증용 파일 목록으로 나눈다.
        train_files = random_train[:train_nums[ii]]
        valid_files = random_train[train_nums[ii]:]
        
        # train 데이터 복사
        for file in train_files :
            # 파일의 이름을 가져온다.
            filename = os.path.basename(file)
            # 경로를 생성한다.
            p_dir = os.path.split(os.path.split(file)[0])[1]
            # 복사한다.
            shutil.copy(file, f'{o_path}/train/{ho}/{int(p_dir)}')
            
        # valid 데이터 복사
        for file in valid_files :
            # 파일의 이름을 가져온다.
            filename = os.path.basename(file)
            # 경로를 생성한다.
            p_dir = os.path.split(os.path.split(file)[0])[1]
            # 복사한다.
            shutil.copy(file, f'{o_path}/valid/{ho}/{int(p_dir)}')
            
print('복사완료')

복사완료


In [11]:
# 레이블이 담겨져 있는 파일을 생성한다.
str = '0:airplanes,1:Motorbikes,2:Faces_easy,3:watch,4:Leopards,5:bonsai'
with open(f'{o_path}/label.csv', 'w') as fp :
    fp.write(str)

# Yolo 용
# 데이터 베치용 root_path

yolo_path = 'data/Yolo'
if not os.path.exists(f'{yolo_path}/train/') :
    os.makedirs(f'{yolo_path}/train/')
if not os.path.exists(f'{yolo_path}/test/') :
    os.makedirs(f'{yolo_path}/test/')
